In [24]:
import numpy as np
import pandas as pd
import scipy as sc

import random

from sklearn.preprocessing import LabelEncoder, StandardScaler

In [25]:
data_init = pd.read_excel("C:/Users/Berezina.Darya/Documents/GitHub/Sberbank/total4.xlsx")

In [2]:
data_init = pd.read_excel("/Users/daber/Projects/Sberbank/total4.xlsx")

In [26]:
cian = pd.read_csv("C:/Users/Berezina.Darya/Documents/GitHub/Sberbank/cian_prices", header=None, sep=";", 
                   names=["sub_area", "avp", "growth"])

In [3]:
cian = pd.read_csv("/Users/daber/Projects/Sberbank/cian_prices", header=None, sep=";", 
                   names=["sub_area", "avp", "growth"])

In [27]:
cian.head()

,sub_area,avp,growth
0,Ajeroport,221 317,0.9
1,Akademicheskoe,213 748,0.4
2,Alekseevskoe,213 848,0.6
3,Altuf'evskoe,151 306,-1.3
4,Arbat,526 173,7.2


In [28]:
cian.avp = cian.avp.apply(lambda x: int(str(x).replace(" ", "")))

In [29]:
data = data_init.copy()
data["price_m"] = data.price_doc / data.full_sq
data["month"] = [int((d[:4])+(d[5:7])) for d in data.timestamp.values]
data["year"] = [int(d[:4]) for d in data.timestamp.values]
data["expl"] = -data.build_year + data.year

In [30]:
to_take = ["id", "full_sq", "life_sq", "floor", "max_floor", "build_year", "num_room",
          "state", "sub_area", 'kitch_sq','metro_min_avto',"month", "material", "price_doc", "price_m",
          "year", "expl", "kindergarten_km", "school_km", "public_transport_station_km"]

In [31]:
df = data[to_take].reset_index(drop=True)

In [32]:
df = df.merge(cian, on="sub_area")

In [33]:
df["coef"] = df.avp / np.unique(df.avp).mean()
df["pma"] = df.price_m / df.coef

In [34]:
df.head()

,id,full_sq,life_sq,floor,max_floor,build_year,num_room,state,sub_area,kitch_sq,...,price_m,year,expl,kindergarten_km,school_km,public_transport_station_km,avp,growth,coef,pma
0,1,43.0,27.0,4.0,9.0,1974.0,2.0,2.0,Bibirevo,6.0,...,136046.511628,2011,37.0,0.145700,0.177975,0.274985,154149,-1.6,0.864001,157460.989574
1,8,44.0,NaN,5.0,9.0,1974.0,2.0,NaN,Bibirevo,6.0,...,45454.545455,2011,37.0,0.134365,0.581511,0.254814,154149,-1.6,0.864001,52609.343836
2,33,38.0,34.0,2.0,NaN,NaN,NaN,NaN,Bibirevo,NaN,...,128947.368421,2011,NaN,0.151205,0.295021,0.054883,154149,-1.6,0.864001,149244.401724
3,71,32.0,19.0,9.0,9.0,1975.0,1.0,2.0,Bibirevo,6.0,...,165625.000000,2011,36.0,0.121817,0.346549,0.167543,154149,-1.6,0.864001,191695.296602
4,103,44.0,28.0,4.0,9.0,1974.0,2.0,NaN,Bibirevo,6.0,...,135227.272727,2011,37.0,0.061154,0.595054,0.227483,154149,-1.6,0.864001,156512.797912


In [83]:
df2 = df.copy()
df2.fillna(0, inplace=True)

In [84]:
cols = ['full_sq', 'life_sq', 'floor', 'max_floor', 'build_year', 'num_room',
       'state', 'kitch_sq', 'metro_min_avto', 'material', 'year', 'expl',
       'kindergarten_km', 'school_km', 'public_transport_station_km', 'avp']

In [85]:
df3 = df2.copy()
for col in cols:
    df3.ix[:,col] = (df3.ix[:,col] - df3.ix[:,col].min())/(df3.ix[:,col].max() - df3.ix[:,col].min())

In [86]:
df4 = df3[df3.price_doc > 0].reset_index(drop=True)
df5 = df4.drop(["id", "sub_area", "price_doc", "price_m", "growth","month", "coef", "pma"],1)

In [87]:
df6 = df3[df3.price_doc == 0].reset_index(drop=True)
df7 = df6.drop(["id", "sub_area", "price_doc", "price_m", "growth","month", "coef", "pma"],1)

In [36]:
#ss = StandardScaler()
#X = ss.fit_transform(df3)

In [88]:
X = df5.values

In [80]:
def find_similar(ind):
    v = X[ind]
    r = []
    for i in range(len(X)):
        if i != ind:
            r.append((i,sc.linalg.norm(X[i]-v)))
    r.sort(key=lambda x: x[1], reverse=False)
    rr = []
    for i in range(10):
        rr.append(df4[["pma"]].loc[r[i][0]].values[0])
    rr.sort()
    
    best, std = 0, np.std(rr[:6])
    for i in range(1,len(rr)-6):
        if np.std(rr[i:i+3]) < std:
            best = i
            std = np.std(rr[i:i+6])

    return np.mean(np.array(rr[best:best+6]))*df4[["coef"]].loc[ind].values[0]

In [89]:
def find_similar(v):
    r = []
    for i in range(len(X)):
        r.append((i,sc.linalg.norm(X[i]-v)))
    r.sort(key=lambda x: x[1], reverse=False)
    rr = []
    for i in range(10):
        rr.append(df4[["pma"]].loc[r[i][0]].values[0])
    rr.sort()
    
    best, std = 0, np.std(rr[:6])
    for i in range(1,len(rr)-6):
        if np.std(rr[i:i+3]) < std:
            best = i
            std = np.std(rr[i:i+6])

    return np.mean(np.array(rr[best:best+6]))*df6[["coef"]].loc[j].values[0]

In [51]:
df2[df2.id == 27449]

,id,full_sq,life_sq,floor,max_floor,build_year,num_room,state,sub_area,kitch_sq,...,price_m,year,expl,kindergarten_km,school_km,public_transport_station_km,avp,growth,coef,pma
2148,27449,77.0,44.0,11.0,14.0,2011.0,3.0,3.0,Kuncevo,8.0,...,38961.038961,2015,4.0,0.076619,0.517121,0.114636,192025,0.3,1.076295,36199.205057


In [52]:
find_similar(2148)

196693.17417682742

In [54]:
39/196-1

-0.8010204081632653

In [294]:
df3.head()

,full_sq,life_sq,floor,max_floor,build_year,num_room,state,kitch_sq,metro_min_avto,material,year,expl,kindergarten_km,school_km,public_transport_station_km,avp
0,43.0,27.0,4.0,9.0,1974.0,2.0,2.0,6.0,2.590241,1.0,2011,37.0,0.145700,0.177975,0.274985,154149
1,44.0,0.0,5.0,9.0,1974.0,2.0,0.0,6.0,3.623270,1.0,2011,37.0,0.134365,0.581511,0.254814,154149
2,38.0,34.0,2.0,0.0,0.0,0.0,0.0,0.0,5.262578,0.0,2011,0.0,0.151205,0.295021,0.054883,154149
3,32.0,19.0,9.0,9.0,1975.0,1.0,2.0,6.0,3.914421,1.0,2011,36.0,0.121817,0.346549,0.167543,154149
4,44.0,28.0,4.0,9.0,1974.0,2.0,0.0,6.0,3.739761,1.0,2011,37.0,0.061154,0.595054,0.227483,154149


In [38]:
wrong = []

for i in range(len(df2)):
    if abs(df2.price_m.values[i] / df2.avp.values[i] - 1) > 0.4:
        wrong.append(1)
    else:
        wrong.append(0)
df2["wrong"] = wrong

In [39]:
inds = random.sample(list(df2[(df2.state > 0)&(df2.build_year > 0)&(df2.max_floor > 0)&(df2.wrong == 0)
                              &(df2.floor > 0)].index), 1000)

In [77]:
inds = list(df5.index)
inds = inds[-1000:]

In [81]:
%%time

p1, p2 = [], []

for ind in inds:
    p1.append(df4[["price_m"]].loc[ind].values[0])
    p2.append(find_similar(ind))

Wall time: 11min 5s


In [82]:
l = (np.log1p(np.array(p1)) - np.log1p(np.array(p2)))**2
np.sqrt((1/len(p1))*sum(l))

0.26668549789603535

In [242]:
0.19443164110479511

0.19948650567017986

In [ ]:
9,5 0.130 0.123 0.129
10,5 0.130 0.128 0.128
10,6 0.129 0.123 0.127

In [15]:
df4 = df.copy()
df4.fillna(0, inplace=True)
df4 = df4[df4.price_doc == 0].reset_index(drop=True)
df5 = df4.drop(["id", "sub_area", "price_doc", "price_m", "growth","month", "coef", "pma"],1)
test_inds = df4.id.values
ss = StandardScaler()
X0 = ss.fit_transform(df5)

In [90]:
X0 = df7.values

In [91]:
%%time

p = []

for j in range(len(X0)):
    v = X0[j]
    p.append(find_similar(v))

Wall time: 1h 22min 20s


In [95]:
df6.head()

,id,life_sq,floor,max_floor,build_year,num_room,state,sub_area,kitch_sq,metro_min_avto,...,year,expl,kindergarten_km,school_km,public_transport_station_km,avp,growth,coef,pma,full_sq
0,30580,0.091810,0.022727,0.157895,0.979693,0.111111,0.50,Bibirevo,0.112281,0.071149,...,0.8,0.257862,0.003328,0.012999,0.003339,0.198426,-1.6,0.864001,0.0,36.6
1,30657,0.129310,0.045455,0.210526,0.978702,0.222222,0.50,Bibirevo,0.117544,0.063280,...,0.8,0.270440,0.003285,0.006421,0.008312,0.198426,-1.6,0.864001,0.0,45.5
2,30897,0.181897,0.204545,0.157895,0.977712,0.333333,0.75,Bibirevo,0.101754,0.030571,...,0.8,0.283019,0.002641,0.006816,0.011958,0.198426,-1.6,0.864001,0.0,59.3
3,31095,0.081034,0.068182,0.157895,0.978207,0.111111,0.50,Bibirevo,0.107018,0.055656,...,0.8,0.276730,0.004604,0.012270,0.014475,0.198426,-1.6,0.864001,0.0,32.6
4,31536,0.170259,0.022727,0.210526,0.978207,0.333333,0.75,Bibirevo,0.140351,0.024186,...,0.8,0.276730,0.004390,0.007530,0.006889,0.198426,-1.6,0.864001,0.0,61.8


In [96]:
len(df6)

7662

In [94]:
df6.drop("full_sq",1,inplace=True)
df6 = df6.merge(df2[["id", "full_sq"]], on="id")

In [97]:
df6["price_m"] = p
df6["price_doc"] = df6.price_m * df6.full_sq

In [99]:
res = df6[["id", "price_doc"]]
res.sort(columns="id", inplace=True)

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app
C:\Anaconda3\lib\site-packages\pandas\core\frame.py:3304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  na_position=na_position)


In [100]:
res.head()

,id,price_doc
2407,30474,6.262847e+06
6929,30475,8.982746e+06
4610,30476,6.269445e+06
6727,30477,8.444016e+06
6930,30478,5.759632e+06


In [101]:
res.to_csv("C:/Users/Berezina.Darya/Documents/GitHub/Sberbank/output.csv", index=False)